# Collect Data From Phaser

In [1]:
import numpy as np
from numpy.fft import fft, fftfreq
import plotly.express as px
import plotly.graph_objects as go
import sys
from configparser import ConfigParser
sys.path.insert(0, "../../chill/.packages/pyadi-iio/") # Path to pyadi-iio library
import adi

## Plotly formatting
Just minimize this cell

In [3]:
PLOTLY_LAYOUT = {
        "font": {
        "family": "JetBrains Mono",
        "size": 18,
        "color": "white",
        },
        'annotationdefaults': {'arrowcolor': '#f2f5fa', 'arrowhead': 0, 'arrowwidth': 1},
                'autotypenumbers': 'strict',
                'coloraxis': {'colorbar': {'outlinewidth': 0, 'ticks': ''}},
                'colorscale': {'diverging': [[0, '#8e0152'], [0.1, '#c51b7d'], [0.2,
                                        '#de77ae'], [0.3, '#f1b6da'], [0.4, '#fde0ef'],
                                        [0.5, '#f7f7f7'], [0.6, '#e6f5d0'], [0.7,
                                        '#b8e186'], [0.8, '#7fbc41'], [0.9, '#4d9221'],
                                        [1, '#276419']],
                        'sequential': [[0.0, '#0d0887'], [0.1111111111111111,
                                        '#46039f'], [0.2222222222222222, '#7201a8'],
                                        [0.3333333333333333, '#9c179e'],
                                        [0.4444444444444444, '#bd3786'],
                                        [0.5555555555555556, '#d8576b'],
                                        [0.6666666666666666, '#ed7953'],
                                        [0.7777777777777778, '#fb9f3a'],
                                        [0.8888888888888888, '#fdca26'], [1.0,
                                        '#f0f921']],
                        'sequentialminus': [[0.0, '#0d0887'], [0.1111111111111111,
                                                '#46039f'], [0.2222222222222222, '#7201a8'],
                                                [0.3333333333333333, '#9c179e'],
                                                [0.4444444444444444, '#bd3786'],
                                                [0.5555555555555556, '#d8576b'],
                                                [0.6666666666666666, '#ed7953'],
                                                [0.7777777777777778, '#fb9f3a'],
                                                [0.8888888888888888, '#fdca26'], [1.0,
                                                '#f0f921']]},
                # 'colorway': [#636efa, #EF553B, #00cc96, #ab63fa, #FFA15A, #19d3f3, #FF6692,
                #             #B6E880, #FF97FF, #FECB52],
                # 'font': {'color': '#f2f5fa'},
                'geo': {'bgcolor': 'rgb(17,17,17)',
                        'lakecolor': 'rgb(17,17,17)',
                        'landcolor': 'rgb(17,17,17)',
                        'showlakes': True,
                        'showland': True,
                        'subunitcolor': '#506784'},
                'hoverlabel': {'align': 'left'},
                'hovermode': 'closest',
                'mapbox': {'style': 'dark'},
                'paper_bgcolor': 'rgb(17,17,17)',
                'plot_bgcolor': 'rgb(17,17,17)',
                'polar': {'angularaxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''},
                        'bgcolor': 'rgb(17,17,17)',
                        'radialaxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''}},
                'scene': {'xaxis': {'backgroundcolor': 'rgb(17,17,17)',
                                'gridcolor': '#506784',
                                'gridwidth': 2,
                                'linecolor': '#506784',
                                'showbackground': True,
                                'ticks': '',
                                'zerolinecolor': '#C8D4E3'},
                        'yaxis': {'backgroundcolor': 'rgb(17,17,17)',
                                'gridcolor': '#506784',
                                'gridwidth': 2,
                                'linecolor': '#506784',
                                'showbackground': True,
                                'ticks': '',
                                'zerolinecolor': '#C8D4E3'},
                        'zaxis': {'backgroundcolor': 'rgb(17,17,17)',
                                'gridcolor': '#506784',
                                'gridwidth': 2,
                                'linecolor': '#506784',
                                'showbackground': True,
                                'ticks': '',
                                'zerolinecolor': '#C8D4E3'}},
                'shapedefaults': {'line': {'color': '#f2f5fa'}},
                'sliderdefaults': {'bgcolor': '#C8D4E3', 'bordercolor': 'rgb(17,17,17)', 'borderwidth': 1, 'tickwidth': 0},
                'ternary': {'aaxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''},
                        'baxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''},
                        'bgcolor': 'rgb(17,17,17)',
                        'caxis': {'gridcolor': '#506784', 'linecolor': '#506784', 'ticks': ''}},
                'title_x': 0.5,
                'updatemenudefaults': {'bgcolor': '#506784', 'borderwidth': 0},
                'xaxis': {'automargin': True,
                        'gridcolor': '#283442',
                        'linecolor': '#506784',
                        'ticks': '',
                        'title': {'standoff': 15},
                        'zerolinecolor': '#283442',
                        'zerolinewidth': 2},
                'yaxis': {'automargin': True,
                        'gridcolor': '#283442',
                        'linecolor': '#506784',
                        'ticks': '',
                        'title': {'standoff': 15},
                        'zerolinecolor': '#283442',
                        'zerolinewidth': 2}
}

In [2]:
config = ConfigParser()
config.read("../app/config.ini")
phaser_config = dict(config["phaser"])
phaser_config

{'rpi_ip': 'ip:192.168.1.108',
 'sdr_ip': 'ip:192.168.2.1',
 'sample_rate_mhz': '0.6',
 'center_freq_mhz': '2100',
 'signal_freq_mhz': '0.1',
 'output_freq_mhz': '12100',
 'bw_mhz': '3',
 'max_gain_bits': '7',
 'num_devs': '2',
 'num_channels': '4',
 'num_slices': '200',
 'gain_control_mode_chan0': 'manual',
 'gain_control_mode_chan1': 'manual',
 'rx_hardware_gain_chan0': '30',
 'rx_hardware_gain_chan1': '30',
 'tx_hardware_gain_chan0': '-88',
 'tx_hardware_gain_chan1': '0',
 'num_steps': '1000',
 'ramp_time_us': '1000',
 'ramp_mode': 'continuous_triangular'}

## Initialize Phaser
Will clean this up later...

In [5]:
my_sdr = adi.ad9361(uri=phaser_config["sdr_ip"])
my_phaser = adi.CN0566(uri=phaser_config["rpi_ip"], rx_dev=my_sdr)

In [6]:
# Initialize both ADAR1000s, set gains to max, and all phases to 0
max_phaser_gain = int(2**int(phaser_config["max_gain_bits"]) - 1)
my_phaser.configure(device_mode="rx")
for i in range(0, 8):
    my_phaser.set_chan_gain(i, max_phaser_gain)
    my_phaser.set_chan_phase(i, 0)


# ADAR1000 devices and channels for each
num_devs = phaser_config["num_devs"]
num_channels = phaser_config["num_channels"]

sample_rate = float(phaser_config["sample_rate_mhz"]) * 1e6
center_freq = float(phaser_config["center_freq_mhz"]) * 1e6
signal_freq = float(phaser_config["signal_freq_mhz"]) * 1e6
num_slices = int(phaser_config["num_slices"])
fft_size = 1024 * 16

# Create radio
'''This script is for Pluto Rev C, dual channel setup'''
my_sdr.sample_rate = int(sample_rate)

# Configure Rx
my_sdr.rx_lo = int(center_freq)   # set this to output_freq - (the freq of the HB100)
my_sdr.rx_enabled_channels = [0, 1]   # enable Rx1 (voltage0) and Rx2 (voltage1)
my_sdr.rx_buffer_size = int(fft_size)
my_sdr.gain_control_mode_chan0 = 'manual'  # manual or slow_attack
my_sdr.gain_control_mode_chan1 = 'manual'  # manual or slow_attack
my_sdr.rx_hardwaregain_chan0 = int(30)   # must be between -3 and 70
my_sdr.rx_hardwaregain_chan1 = int(30)   # must be between -3 and 70
# Configure Tx
my_sdr.tx_lo = int(center_freq)
my_sdr.tx_enabled_channels = [0, 1]
my_sdr.tx_cyclic_buffer = True      # must set cyclic buffer to true for the tdd burst mode.  Otherwise Tx will turn on and off randomly
my_sdr.tx_hardwaregain_chan0 = -88   # must be between 0 and -88
my_sdr.tx_hardwaregain_chan1 = -0   # must be between 0 and -88

# Enable TDD logic in pluto (this is for synchronizing Rx Buffer to ADF4159 TX input)
#gpio = adi.one_bit_adc_dac(sdr_ip)
#gpio.gpio_phaser_enable = True

# Configure the ADF4159 Rampling PLL
output_freq = 12.1e9
c = 3e8
wavelength = c / output_freq
BW = 500e6
num_steps = 1000
ramp_time = 1e3 # us
ramp_time_s = ramp_time / 1e6
my_phaser.frequency = int(output_freq/4) # Output frequency divided by 4
my_phaser.freq_dev_range = int(BW/4) # frequency deviation range in Hz.  This is the total freq deviation of the complete freq ramp
my_phaser.freq_dev_step = int(BW/num_steps) # frequency deviation step in Hz.  This is fDEV, in Hz.  Can be positive or negative
my_phaser.freq_dev_time = int(ramp_time) # total time (in us) of the complete frequency ramp
my_phaser.delay_word = 4095     # 12 bit delay word.  4095*PFD = 40.95 us.  For sawtooth ramps, this is also the length of the Ramp_complete signal
my_phaser.delay_clk = 'PFD'     # can be 'PFD' or 'PFD*CLK1'
my_phaser.delay_start_en = 0         # delay start
my_phaser.ramp_delay_en = 0          # delay between ramps.  
my_phaser.trig_delay_en = 0          # triangle delay
# Enable ADF4159 TX input and generate a single triangular ramp with each trigger
my_phaser.ramp_mode = "continuous_triangular" # ramp_mode can be:  "disabled", "continuous_sawtooth", "continuous_triangular", "single_sawtooth_burst", "single_ramp_burst"
my_phaser.sing_ful_tri = 0           # full triangle enable/disable -- this is used with the single_ramp_burst mode
my_phaser.tx_trig_en = 0             # start a ramp with TXdata
my_phaser.enable = 0                 # 0 = PLL enable.  Write this last to update all the registers

Cal = true, setting channel x to gain y, gcal value:  0 ,  127 ,  1.0
reading back:  -0.127
Cal = true, setting channel x to gain y, gcal value:  1 ,  127 ,  1.0
reading back:  -0.127
Cal = true, setting channel x to gain y, gcal value:  2 ,  127 ,  1.0
reading back:  -0.127
Cal = true, setting channel x to gain y, gcal value:  3 ,  127 ,  1.0
reading back:  -0.127
Cal = true, setting channel x to gain y, gcal value:  4 ,  127 ,  1.0
reading back:  -0.127
Cal = true, setting channel x to gain y, gcal value:  5 ,  127 ,  1.0
reading back:  -0.127
Cal = true, setting channel x to gain y, gcal value:  6 ,  127 ,  1.0
reading back:  -0.127
Cal = true, setting channel x to gain y, gcal value:  7 ,  127 ,  1.0
reading back:  -0.127


## Transmission

### Create signal

In [7]:
fs = int(my_sdr.sample_rate)
ts = 1 / float(fs)
N = int(my_sdr.rx_buffer_size)
fc = int(signal_freq / (fs / N)) * (fs / N)
t = np.arange(0, N * ts, ts)

i = np.cos(2 * np.pi * t * fc) * 2 ** 14
q = np.sin(2 * np.pi * t * fc) * 2 ** 14
iq = 1 * (i + 1j * q)

### Transmit signal

In [22]:
my_sdr._ctx.set_timeout(60000)
my_sdr.tx([iq*0.5, iq])  # only send data to the 2nd channel (that's all we need)
signal = my_sdr.rx()
my_sdr.tx_destroy_buffer()
signal = np.array(signal).sum(axis=0)
signal.shape

(16384,)

## Simple processing 

In [23]:
signal_fft = np.abs(fft(signal))
signal_fft /= signal_fft.max()
freq = fftfreq(N, d=ts)

## Plotting

In [24]:
fig = px.line(x=freq, y=np.log10(np.abs(signal_fft)))
fig.update_layout(PLOTLY_LAYOUT)
fig.update_layout(
    title="Signal FFT",
    xaxis=dict(title="Frequency (Hz)"),
    yaxis=dict(title="Magnitude (dB)"),
)
fig.show()